In [72]:
import os
import csv
import pandas as pd
import numpy as np
#load datasets

airbnb = pd.read_csv('AB_NYC_2019.csv')
crime = pd.read_csv('NYC_crime.csv')

In [73]:
#Visualize data
print(airbnb)
print(crime)

             id                                               name   host_id  \
0          2539                 Clean & quiet apt home by the park      2787   
1          2595                              Skylit Midtown Castle      2845   
2          3647                THE VILLAGE OF HARLEM....NEW YORK !      4632   
3          3831                    Cozy Entire Floor of Brownstone      4869   
4          5022   Entire Apt: Spacious Studio/Loft by central park      7192   
...         ...                                                ...       ...   
48890  36484665    Charming one bedroom - newly renovated rowhouse   8232441   
48891  36485057      Affordable room in Bushwick/East Williamsburg   6570630   
48892  36485431            Sunny Studio at Historical Neighborhood  23492952   
48893  36485609               43rd St. Time Square-cozy single bed  30985759   
48894  36487245  Trendy duplex in the very heart of Hell's Kitchen  68119814   

           host_name neighbourhood_grou

In [74]:
#Check null values
print(airbnb.isna().sum())
print(crime.isna().sum())

#Drop rows with null values
airbnb = airbnb.dropna()
crime = crime.dropna()


id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64
Unnamed: 0                         0
arrest_key                         0
arrest_date                        0
pd_desc                            0
ofns_desc                          0
law_code                           0
law_cat_cd                     13360
age_group                          0
perp_sex                           0
perp_race     

In [75]:
#Rename columns for consistency
crime = crime.rename(columns={':@computed_region_f5dn_yrer': 'community_districts', 
                              ':@computed_region_yeji_bk3q': 'borough_boundaries',  
                              ':@computed_region_92fq_4b7q': 'city_council_districts',
                              ':@computed_region_sbqj_enih': 'police_precincts',
                              })

In [76]:
#Check unique values in order to decide wich columns to keep in crime dataset
print(airbnb['neighbourhood_group'].unique())
print(airbnb['neighbourhood'].unique())

['Brooklyn' 'Manhattan' 'Queens' 'Staten Island' 'Bronx']
['Kensington' 'Midtown' 'Clinton Hill' 'East Harlem' 'Murray Hill'
 'Bedford-Stuyvesant' "Hell's Kitchen" 'Upper West Side' 'Chinatown'
 'South Slope' 'West Village' 'Williamsburg' 'Fort Greene' 'Chelsea'
 'Crown Heights' 'Park Slope' 'Windsor Terrace' 'Inwood' 'East Village'
 'Harlem' 'Greenpoint' 'Bushwick' 'Lower East Side'
 'Prospect-Lefferts Gardens' 'Long Island City' 'Kips Bay' 'SoHo'
 'Upper East Side' 'Prospect Heights' 'Washington Heights' 'Woodside'
 'Flatbush' 'Brooklyn Heights' 'Carroll Gardens' 'Gowanus' 'Flatlands'
 'Cobble Hill' 'Flushing' 'Boerum Hill' 'Sunnyside' 'DUMBO' 'St. George'
 'Highbridge' 'Financial District' 'Ridgewood' 'Morningside Heights'
 'Jamaica' 'Middle Village' 'NoHo' 'Ditmars Steinway' 'Flatiron District'
 'Roosevelt Island' 'Greenwich Village' 'Little Italy' 'East Flatbush'
 'Tompkinsville' 'Astoria' 'Eastchester' 'Kingsbridge' 'Two Bridges'
 'Rockaway Beach' 'Forest Hills' 'Nolita' 'Woodlaw

In [77]:
#Check Columns 
print(airbnb.columns)
print(crime.columns)

# Drop columns that are not needed and exist in the dataframe
airbnb = airbnb.drop(columns=['name','host_id', 'host_name','neighbourhood'])
crime = crime[['arrest_date', 'ofns_desc', 'law_cat_cd', 'latitude', 'longitude','borough_boundaries']]

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')
Index(['Unnamed: 0', 'arrest_key', 'arrest_date', 'pd_desc', 'ofns_desc',
       'law_code', 'law_cat_cd', 'age_group', 'perp_sex', 'perp_race',
       'latitude', 'longitude', 'arrest_boro', 'arrest_precinct',
       'jurisdiction_code', 'community_districts', 'borough_boundaries',
       'city_council_districts', 'police_precincts'],
      dtype='object')


In [78]:
#Check data types
print(airbnb.dtypes)
print(crime.dtypes)

id                                  int64
neighbourhood_group                object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object
arrest_date            object
ofns_desc              object
law_cat_cd             object
latitude              float64
longitude             float64
borough_boundaries    float64
dtype: object


In [79]:
# Rename column to match the neighbourhood group in airnb dataset
crime = crime.rename(columns={'borough_boundaries': 'neighbourhood_group'})

# Map the neighbourhood_group to the crime dataset
crime['neighbourhood_group'] = crime['neighbourhood_group'].map({
    1.0: 'Manhattan',
    2.0: 'Bronx',
    3.0: 'Brooklyn',
    4.0: 'Queens',
    5.0: 'Staten Island'
})

In [80]:
#Coverting the 'arrest_date' and 'last review' to datetime format
crime['arrest_date'] = pd.to_datetime(crime['arrest_date'], format='%Y-%m-%d')
airbnb['last_review'] = pd.to_datetime(airbnb['last_review'], format='%Y-%m-%d')

In [81]:
#Find a threshold for arrest_date 
print(airbnb['last_review'].describe())
print(crime['arrest_date'].describe())


count                            38821
mean     2018-10-04 07:23:47.072975872
min                2011-03-28 00:00:00
25%                2018-07-09 00:00:00
50%                2019-05-19 00:00:00
75%                2019-06-23 00:00:00
max                2019-07-08 00:00:00
Name: last_review, dtype: object
count                          3862648
mean     2012-05-12 13:20:44.880300288
min                2006-01-01 00:00:00
25%                2009-02-20 00:00:00
50%                2012-02-22 00:00:00
75%                2015-06-06 00:00:00
max                2019-12-31 00:00:00
Name: arrest_date, dtype: object


In [82]:
# Filter the crime dataset to only include arrests after a specific date
crime = crime[crime['arrest_date'] >= pd.Timestamp('2011-03-28')]

In [83]:
#chech duplicates
print(airbnb.duplicated().sum())
print(crime.duplicated().sum())

0
325147


In [84]:
#Remove duplicates
crime = crime.drop_duplicates()

In [85]:
# Aggregate crime data by neighbourhood_group
crime_grouped = crime.groupby('neighbourhood_group').size().reset_index(name='crime_count')

# Merge the aggregated crime data with airnb
airbnb_crime = pd.merge(airbnb, crime_grouped, on='neighbourhood_group', how='inner')

In [86]:
# Attractions with coordinates
nyc_attractions = {
    "Statue of Liberty": (40.689247, -74.044502),
    "Times Square": (40.758896, -73.985130),
    "Central Park": (40.785091, -73.968285),
    "Empire State Building": (40.748817, -73.985428),
    "Brooklyn Bridge": (40.706086, -73.996864),
    "Rockefeller Center": (40.758740, -73.978674),
    "One World Trade Center": (40.712742, -74.013382),
    "Broadway": (40.759011, -73.984472),
    "Grand Central Terminal": (40.752726, -73.977229),
    "The Metropolitan Museum of Art": (40.779437, -73.963244),
    "American Museum of Natural History": (40.781324, -73.973988),
    "9/11 Memorial & Museum": (40.711503, -74.013379),
    "Fifth Avenue": (40.775391, -73.965633),
    "Chrysler Building": (40.751652, -73.975311),
    "The High Line": (40.747992, -74.004764)
}

# Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Loop through attractions and add new distance columns
for name, (lat, lon) in nyc_attractions.items():
    col_name = "distance_to_" + name.lower().replace(" ", "_").replace("&", "and") + "_km"
    airbnb_crime[col_name] = airbnb_crime.apply(lambda row: haversine(row["latitude"], row["longitude"], lat, lon), axis=1)

print(airbnb_crime.head())


     id neighbourhood_group  latitude  longitude        room_type  price  \
0  2539            Brooklyn  40.64749  -73.97237     Private room    149   
1  2595           Manhattan  40.75362  -73.98377  Entire home/apt    225   
2  3831            Brooklyn  40.68514  -73.95976  Entire home/apt     89   
3  5022           Manhattan  40.79851  -73.94399  Entire home/apt     80   
4  5099           Manhattan  40.74767  -73.97500  Entire home/apt    200   

   minimum_nights  number_of_reviews last_review  reviews_per_month  ...  \
0               1                  9  2018-10-19               0.21  ...   
1               1                 45  2019-05-21               0.38  ...   
2               1                270  2019-07-05               4.64  ...   
3              10                  9  2018-11-19               0.10  ...   
4               3                 74  2019-06-22               0.59  ...   

   distance_to_rockefeller_center_km  distance_to_one_world_trade_center_km  \
0      

In [87]:
airbnb_crime

,id,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,...,distance_to_rockefeller_center_km,distance_to_one_world_trade_center_km,distance_to_broadway_km,distance_to_grand_central_terminal_km,distance_to_the_metropolitan_museum_of_art_km,distance_to_american_museum_of_natural_history_km,distance_to_9/11_memorial_and_museum_km,distance_to_fifth_avenue_km,distance_to_chrysler_building_km,distance_to_the_high_line_km
0,2539,Brooklyn,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,...,12.381844,8.037748,12.442461,11.708876,14.691985,14.882287,7.913507,14.233274,11.584939,11.504155
1,2595,Manhattan,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,...,0.712997,5.185197,0.602361,0.559866,3.350997,3.188791,5.306275,2.862444,0.745382,1.875920
2,3831,Brooklyn,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,...,8.337723,5.463919,8.473967,7.658067,10.489457,10.762152,5.387559,10.047648,7.511024,7.951725
3,5022,Manhattan,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,...,5.299525,11.185555,5.559599,5.809606,2.669340,3.166975,11.303146,3.150993,5.839852,7.599008
4,5099,Manhattan,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,...,1.269235,5.054090,1.492264,0.592728,3.668465,3.743125,5.160574,3.181790,0.443553,2.507579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38816,36425863,Manhattan,40.78099,-73.95366,Private room,129,1,1,2019-07-07,1.00,...,3.249367,9.105050,3.564490,3.717130,0.825234,1.711978,9.220080,1.184891,3.737230,5.655697
38817,36427429,Queens,40.75104,-73.81459,Private room,45,1,1,2019-07-07,1.00,...,13.847483,17.283137,14.336755,13.701176,12.911414,13.839918,17.317505,13.005918,13.538611,16.023458
38818,36438336,Staten Island,40.54179,-74.14275,Private room,235,1,1,2019-07-07,1.00,...,27.812877,21.921177,27.599060,27.297368,30.455700,30.200240,21.801995,29.965427,27.278328,25.714890
38819,36442252,Bronx,40.80787,-73.92400,Entire home/apt,100,1,2,2019-07-07,2.00,...,7.143857,12.983129,7.445728,7.595059,4.572722,5.140092,13.095514,5.032590,7.598851,9.517212


In [88]:
# Save the cleaned datasets to CSV files
airbnb_crime.to_csv('cleaned_airnb_crime.csv', index=False)

In [89]:
#check columns
print(airbnb_crime.columns)

Index(['id', 'neighbourhood_group', 'latitude', 'longitude', 'room_type',
       'price', 'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'crime_count', 'distance_to_statue_of_liberty_km',
       'distance_to_times_square_km', 'distance_to_central_park_km',
       'distance_to_empire_state_building_km',
       'distance_to_brooklyn_bridge_km', 'distance_to_rockefeller_center_km',
       'distance_to_one_world_trade_center_km', 'distance_to_broadway_km',
       'distance_to_grand_central_terminal_km',
       'distance_to_the_metropolitan_museum_of_art_km',
       'distance_to_american_museum_of_natural_history_km',
       'distance_to_9/11_memorial_and_museum_km',
       'distance_to_fifth_avenue_km', 'distance_to_chrysler_building_km',
       'distance_to_the_high_line_km'],
      dtype='object')
